# TP RNN : Sentiment calssification
Binome :
- Hafdi Ramy
- Chaalal Med. Elmondhir

# Importing the libraries

In [ ]:
import pandas as pd
import tensorflow as tf
import os
import glob
import re
from random import shuffle
from gensim.models import Word2Vec

import numpy as np
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Downloading the dataset

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')


84125825/84125825 [==============================] - 30s 0us/step


In [ ]:
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

In [ ]:
train_dir = os.path.join(dataset_dir, 'train')

In [ ]:
# remove the "unsup" folder to respect the folder structure
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [ ]:
# Create validation and training sets from the training folder
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset="training", 
    seed=seed)

raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset="validation", 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
# create test dataset 
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.


## Text pre-processing


In [ ]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

### Create the pre-processing layer

In [ ]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [ ]:
# remove labels 
train_text = raw_train_ds.map(lambda x, y: x)

# fit the layer using the adapt method
vectorize_layer.adapt(train_text)

In [ ]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

### Applying pre-processing to all layers

In [ ]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

### pipeline optimization

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Model 1: Simple RNN

In [ ]:
embed_size = 128
model = tf.keras.models.Sequential([
    layers.Embedding(max_features + 1, embed_size,
    input_shape=[None]),
    layers.Dropout(0.2),
    # layers.GRU(32, return_sequences=True),
    layers.SimpleRNN(32),
    layers.Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          640064    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                3104      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 643,201
Trainable params: 643,201
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
625/625 [==============================] - 145s 227ms/step - loss: 0.6950 - accuracy: 0.5020 - val_loss: 0.6936 - val_accuracy: 0.5078
Epoch 2/10
625/625 [==============================] - 136s 217ms/step - loss: 0.6422 - accuracy: 0.5972 - val_loss: 0.7241 - val_accuracy: 0.5014
Epoch 3/10
447/625 [====================>.........] - ETA: 38s - loss: 0.5488 - accuracy: 0.6505

KeyboardInterrupt: ignored

The SimpleRNN layer has a major flaw, which is how time consuming it is compared to it's result, so it's rarely used in modern NLP task, compared to more recent layers such as LSTM and GRU

## Model 2: GRU

In [ ]:
embed_size = 128
model = tf.keras.models.Sequential([
    layers.Embedding(max_features + 1, embed_size,
    input_shape=[None]),
    layers.Dropout(0.2),
    # layers.GRU(32, return_sequences=True),
    layers.GRU(32),
    layers.Dropout(0.2),
    layers.Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 128)         1280128   
                                                                 
 dropout_2 (Dropout)         (None, None, 128)         0         
                                                                 
 gru_2 (GRU)                 (None, 32)                15552     
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,295,713
Trainable params: 1,295,713
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
625/625 [==============================] - 10s 13ms/step - loss: 0.6929 - accuracy: 0.5070 - val_loss: 0.6924 - val_accuracy: 0.5146
Epoch 2/10
625/625 [==============================] - 8s 14ms/step - loss: 0.6671 - accuracy: 0.5671 - val_loss: 0.6897 - val_accuracy: 0.5194
Epoch 3/10
625/625 [==============================] - 8s 12ms/step - loss: 0.6013 - accuracy: 0.6321 - val_loss: 0.5562 - val_accuracy: 0.7678
Epoch 4/10
625/625 [==============================] - 8s 12ms/step - loss: 0.4716 - accuracy: 0.7930 - val_loss: 0.6607 - val_accuracy: 0.6062
Epoch 5/10
625/625 [==============================] - 8s 12ms/step - loss: 0.3556 - accuracy: 0.8593 - val_loss: 0.4019 - val_accuracy: 0.8334
Epoch 6/10
625/625 [==============================] - 8s 13ms/step - loss: 0.2208 - accuracy: 0.9208 - val_loss: 0.3650 - val_accuracy: 0.8560
Epoch 7/10
625/625 [==============================] - 8s 12ms/step - loss: 0.1420 - accuracy: 0.9546 - val_loss: 0.3911 - val_accuracy: 0.854

## Model 3: Bi-directional GRU

In [ ]:
embed_size = 128
model = tf.keras.models.Sequential([
    layers.Embedding(max_features + 1, embed_size,
    input_shape=[None]),
    layers.Dropout(0.2),
    # layers.GRU(32, return_sequences=True),
    layers.Bidirectional(layers.GRU(32)),
    layers.Dropout(0.2),
    layers.Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          640064    
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               18816     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 658,945
Trainable params: 658,945
Non-trainable params: 0
________________________________________________

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
625/625 [==============================] - 16s 22ms/step - loss: 0.4883 - accuracy: 0.7537 - val_loss: 0.3559 - val_accuracy: 0.8548
Epoch 2/10
625/625 [==============================] - 12s 19ms/step - loss: 0.2821 - accuracy: 0.8898 - val_loss: 0.3506 - val_accuracy: 0.8588
Epoch 3/10
625/625 [==============================] - 12s 20ms/step - loss: 0.2160 - accuracy: 0.9222 - val_loss: 0.4138 - val_accuracy: 0.8092
Epoch 4/10
625/625 [==============================] - 13s 20ms/step - loss: 0.1668 - accuracy: 0.9412 - val_loss: 0.3720 - val_accuracy: 0.8410
Epoch 5/10
625/625 [==============================] - 13s 21ms/step - loss: 0.1300 - accuracy: 0.9582 - val_loss: 0.4587 - val_accuracy: 0.8238
Epoch 6/10
625/625 [==============================] - 12s 20ms/step - loss: 0.0951 - accuracy: 0.9695 - val_loss: 0.4679 - val_accuracy: 0.8512
Epoch 7/10
625/625 [==============================] - 13s 21ms/step - loss: 0.0854 - accuracy: 0.9722 - val_loss: 0.4837 - val_accuracy: